# Description
This notebook compares the results of training a Conditional Normalizing Flow model on the synthetic two moons data. The data have rotation and moon class as conditioning variables.

The comparison is between a base flow with no regularization at all and adding in Clipped Adam and Batchnormalization.

Lots of different hyperparamters are tested to see if we can get batch normalization to work

# Base loads

In [59]:
!pip install pyro-ppl==1.3.0

In [60]:
!nvidia-smi

Wed Jun 10 11:41:32 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P0    59W / 149W |   2475MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
# "Standard" imports
import numpy as np
from time import time
import itertools
import matplotlib.pyplot as plt
import pickle
import os
import pandas as pd
import folium
import datetime

# Pytorch imports
import torch
import torch.nn.functional as F
from torch import nn
from torch import optim
from torch.utils.data import DataLoader

# Pyro imports
import pyro
from pyro.distributions import ConditionalTransformedDistribution, ConditionalTransformModule, TransformModule
import pyro.distributions as dist
from pyro.distributions.transforms import affine_coupling, affine_autoregressive, permute

# Sklearn imports
from sklearn import datasets
from sklearn.preprocessing import StandardScaler

# Notebooks imports
from IPython.display import Image, display, clear_output
from tqdm import tqdm
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets


import numpy as np
import folium
from folium import plugins
import matplotlib
import seaborn as sns

In [62]:
# Mount my drive
from google.colab import drive
import sys
import os
drive.mount('/content/drive/')
root_path = 'drive/My Drive/Colab_Notebooks/normalizingflows'
trained_flows_folder = 'drive/My Drive/Colab_Notebooks/normalizingflows/trained_flows'
plot_folder = f'{root_path}/plot_notebooks/generated_plots'
dataset_folder = 'drive/My Drive/Colab_Notebooks/normalizingflows/datasets'
sys.path.append(root_path)

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


## Load Git code

In [63]:
%cd drive/'My Drive'/Thesis/code/DEwNF/
!git pull
%cd /content/

/content/drive/My Drive/Thesis/code/DEwNF
Already up to date.
/content


In [0]:
git_folder_path = 'drive/My Drive/Thesis/code/DEwNF/'
sys.path.append(git_folder_path)

In [0]:
from DEwNF.flows import ConditionalAffineCoupling, ConditionedAffineCoupling, ConditionalNormalizingFlowWrapper, conditional_affine_coupling, normalizing_flow_factory, conditional_normalizing_flow_factory
from DEwNF.utils import plot_4_contexts_cond_flow, plot_loss, sliding_plot_loss, plot_samples, simple_data_split_conditional, simple_data_split, circle_transform
from DEwNF.samplers import RotatingTwoMoonsConditionalSampler
from DEwNF.regularizers import NoiseRegularizer, rule_of_thumb_noise_schedule, approx_rule_of_thumb_noise_schedule, square_root_noise_schedule, constant_regularization_schedule


# Func def

In [0]:
def minMax(x):
    return pd.Series(index=['min','max'],data=[x.min(),x.max()])

def rescale_samples(plot_flow_dist, scaler, n_samples=256):
    x_s = plot_flow_dist.sample((n_samples,))
    if x_s.is_cuda:
        x_s = x_s.cpu()
    if scaler is not None:
        x_s = scaler.inverse_transform(x_s)
    return x_s

def add_marker_w_prob(coords, obs_scaler, flow_dist, hub_map, name):
    scaled_coords = obs_scaler.transform(np.flip(np.array([coords])))
    log_prob = flow_dist.log_prob(torch.tensor(scaled_coords).float().cuda()).cpu().detach().numpy()
    folium.CircleMarker(
    location=coords,
    popup=folium.Popup(f'{name} log likelihood: {log_prob}', show=False),
    icon=folium.Icon(icon='cloud')
    ).add_to(hub_map)

def create_overlay(shape, bounds, flow_dist, hub_map, cm, flip):
    with torch.no_grad():
        nlats, nlons = shape
        
        lats_array = torch.linspace(start=bounds[1][0], end=bounds[0][0], steps=nlats)
        lons_array = torch.linspace(start=bounds[0][1], end=bounds[1][1], steps=nlons)
        x, y = torch.meshgrid(lats_array, lons_array)

        points = torch.stack((x.reshape(-1), y.reshape(-1)), axis=1)

        if flip:
            points = points.flip(1)

        scaled_points = torch.tensor(obs_scaler.transform(points), requires_grad=False).float()

        data = flow_dist.log_prob(scaled_points.cuda()).reshape(nlats,nlons).cpu().detach().numpy()
        data = np.exp(data)
        overlay = cm(data)

    folium.raster_layers.ImageOverlay(
    image=overlay,
    bounds=bounds,
    mercator_project=False,
    opacity=0.75
    ).add_to(hub_map)

    return overlay


def plot_samples(samples, hub_map):

    for point in samples:
        folium.CircleMarker(
        location=point,
        radius=2,
        color='#3186cc',
        fill=True,
        fill_color='#3186cc',
        opacityb=0.1).add_to(hub_map)

    return hub_map

# Path plots

In [0]:
nyc_data_small_path = f"{dataset_folder}/NYC_yellow_taxi/processed_nyc_taxi_small.csv"
nyc_data_small = pd.read_csv(nyc_data_small_path)
obs_cols = ['dropoff_longitude', 'dropoff_latitude']
context_cols = ['pickup_longitude', 'pickup_latitude',  'pickup_dow_sin', 'pickup_dow_cos',
                 'pickup_tod_sin', 'pickup_tod_cos']
train_dataloader, test_dataloader, obs_scaler, context_scaler= simple_data_split_conditional(df=nyc_data_small,
                                                                  obs_cols=obs_cols,
                                                                  context_cols=context_cols,
                                                                  batch_size=50000,
                                                                  cuda_exp=True)

In [0]:
input_hour = 18
input_dayofweek = 1
input_pickup_lon =-73.983313
input_pickup_lat = 40.733348

In [0]:
def plot_func(hour, dayofweek, pickup_lat, pickup_lon, flow):
    # Made for the context order:
    #['pickup_longitude', 'pickup_latitude', 'pickup_dow_sin', 'pickup_dow_cos', 'pickup_tod_sin, 'pickup_tod_cos']

    flow.modules.eval()

    pickup_tod_cos, pickup_tod_sin = circle_transform(hour,23)
    pickup_dow_cos, pickup_dow_sin = circle_transform(dayofweek,6)

    context = np.array([[pickup_lon, pickup_lat, pickup_dow_sin, pickup_dow_cos, pickup_tod_sin, pickup_tod_cos]])
    print(context)

    scaled_context = torch.tensor(context_scaler.transform(context)).type(torch.FloatTensor).cuda()
    #scaled_context = torch.tensor([-0.4450,  0.1878, -1.2765, -0.8224,  1.4222, -0.8806]).float().cuda()

    cond_dist = flow.condition(scaled_context)

        
    x_bounds = [-73.95, -73.75]
    y_bounds = [40.62, 40.86]

    x_loc = np.mean(x_bounds)
    y_loc = np.mean(y_bounds)

    lower_left = [y_bounds[0], x_bounds[0]]
    upper_right = [y_bounds[1], x_bounds[1]]
    bounds = [lower_left, upper_right]
    #initial_location = [40.739999999999995, -73.85]
    initial_location = [40.720999999999995, -73.855]
    hub_map = folium.Map(width=550,height=650,location=initial_location, zoom_start=12, zoom_control=False)


    cm = sns.cubehelix_palette(reverse=True, dark=0, light=1, gamma=0.8, as_cmap=True)

    create_overlay(shape=(800,800), bounds=bounds, flow_dist=cond_dist, hub_map=hub_map, cm=cm, flip=True)

    #samples = rescale_samples(cond_dist, obs_scaler, n_samples=1000)
    #plot_samples(samples, hub_map)


    folium.CircleMarker(
    location=[pickup_lat, pickup_lon],
    radius=2,
    color='#3186cc',
    fill=True,
    fill_color='#3186cc',
    opacityb=0.1).add_to(hub_map)

    # add log prob at JFC
    jfc_coords = [40.645587,-73.787536]
    add_marker_w_prob(coords=jfc_coords, obs_scaler=obs_scaler, flow_dist=cond_dist, hub_map=hub_map, name='JFC')

    # add log prob at LaGuardia
    lg_coords = [40.774277,-73.874258]
    add_marker_w_prob(coords=lg_coords, obs_scaler=obs_scaler, flow_dist=cond_dist, hub_map=hub_map, name='LaGuardia')

    train_station1 = [40.6586078,-73.8052761]
    folium.CircleMarker(
    location=train_station1,
    radius=2,
    color='#3186cc',
    fill=True,
    fill_color='#3186cc',
    opacityb=0.1).add_to(hub_map)

    train_station2 = [40.6612707,-73.8294946]
    folium.CircleMarker(
    location=train_station2,
    radius=2,
    color='#3186cc',
    fill=True,
    fill_color='#3186cc',
    opacityb=0.1).add_to(hub_map)


    display(hub_map)
    return hub_map

In [70]:
AC_flow_path = os.path.join(root_path, "results/nyc_taxi/conditional/test3/run1/nyc_model2.pickle")

with open(AC_flow_path, 'rb') as f:
    loaded_dict = pickle.load(f)

ac_flow = loaded_dict['model']



hub_map = plot_func(hour=input_hour, dayofweek=input_dayofweek, pickup_lat=input_pickup_lat, pickup_lon=input_pickup_lon, flow=ac_flow)

hub_map.save(f'{plot_folder}/paper_ac_path.html')


[[-73.983313    40.733348     0.8660254    0.5         -0.97908409
    0.20345601]]


In [71]:
planar_flow_path = os.path.join(root_path, "results/nyc_taxi/planar/test3_planar3/run3/nyc_model_planar2.pickle")

with open(planar_flow_path, 'rb') as f:
    loaded_dict = pickle.load(f)

planar_flow = loaded_dict['model']



hub_map = plot_func(hour=input_hour, dayofweek=input_dayofweek, pickup_lat=input_pickup_lat, pickup_lon=input_pickup_lon, flow=planar_flow)

hub_map.save(f'{plot_folder}/paper_planar_path.html')

[[-73.983313    40.733348     0.8660254    0.5         -0.97908409
    0.20345601]]


In [72]:
combi_flow_path = os.path.join(root_path, "results/nyc_taxi/combi/test_combi_fixed_bn/run3/nyc_model_combi.pickle")

with open(combi_flow_path, 'rb') as f:
    loaded_dict = pickle.load(f)

combi_flow = loaded_dict['model']



hub_map = plot_func(hour=input_hour, dayofweek=input_dayofweek, pickup_lat=input_pickup_lat, pickup_lon=input_pickup_lon, flow=combi_flow)

hub_map.save(f'{plot_folder}/paper_combi_path.html')

[[-73.983313    40.733348     0.8660254    0.5         -0.97908409
    0.20345601]]


# Overview plots

In [0]:
def plot_func(hour, dayofweek, pickup_lat, pickup_lon, flow):
    # Made for the context order:
    #['pickup_longitude', 'pickup_latitude', 'pickup_dow_sin', 'pickup_dow_cos', 'pickup_tod_sin, 'pickup_tod_cos']

    flow.modules.eval()

    pickup_tod_cos, pickup_tod_sin = circle_transform(hour,23)
    pickup_dow_cos, pickup_dow_sin = circle_transform(dayofweek,6)

    context = np.array([[pickup_lon, pickup_lat, pickup_dow_sin, pickup_dow_cos, pickup_tod_sin, pickup_tod_cos]])
    print(context)

    scaled_context = torch.tensor(context_scaler.transform(context)).type(torch.FloatTensor).cuda()
    #scaled_context = torch.tensor([-0.4450,  0.1878, -1.2765, -0.8224,  1.4222, -0.8806]).float().cuda()

    cond_dist = flow.condition(scaled_context)

    x_bounds = [-74.04, -73.75]
    y_bounds = [40.62, 40.86]

    x_loc = np.mean(x_bounds)
    y_loc = np.mean(y_bounds)

    lower_left = [y_bounds[0], x_bounds[0]]
    upper_right = [y_bounds[1], x_bounds[1]]
    bounds = [lower_left, upper_right]
    initial_location = [y_loc, x_loc]
    hub_map = folium.Map(width=900,height=1300,location=initial_location, zoom_start=12, zoom_control=False)


    cm = sns.cubehelix_palette(reverse=True, dark=0, light=1, gamma=0.8, as_cmap=True)

    create_overlay(shape=(800,800), bounds=bounds, flow_dist=cond_dist, hub_map=hub_map, cm=cm, flip=True)

    #samples = rescale_samples(cond_dist, obs_scaler, n_samples=1000)
    #plot_samples(samples, hub_map)

    rectangle_x = [-73.95, -73.765018]
    rectangle_y = [40.6287401, 40.7829614]
    rectangle_lower_left = [rectangle_y[0], rectangle_x[0]]
    rectangle_upper_right = [rectangle_y[1], rectangle_x[1]]
    bounds = [rectangle_lower_left, rectangle_upper_right]

    folium.Rectangle(bounds=bounds, color='red', fill=True, fill_color='red', fill_opacity=0.025).add_to(hub_map)

    display(hub_map)
    return hub_map

In [82]:
AC_flow_path = os.path.join(root_path, "results/nyc_taxi/conditional/test3/run1/nyc_model2.pickle")

with open(AC_flow_path, 'rb') as f:
    loaded_dict = pickle.load(f)

ac_flow = loaded_dict['model']



hub_map = plot_func(hour=input_hour, dayofweek=input_dayofweek, pickup_lat=input_pickup_lat, pickup_lon=input_pickup_lon, flow=ac_flow)

hub_map.save(f'{plot_folder}/paper_ac_overview.html')


[[-73.983313    40.733348     0.8660254    0.5         -0.97908409
    0.20345601]]


In [83]:
planar_flow_path = os.path.join(root_path, "results/nyc_taxi/planar/test3_planar3/run3/nyc_model_planar2.pickle")

with open(planar_flow_path, 'rb') as f:
    loaded_dict = pickle.load(f)

planar_flow = loaded_dict['model']



hub_map = plot_func(hour=input_hour, dayofweek=input_dayofweek, pickup_lat=input_pickup_lat, pickup_lon=input_pickup_lon, flow=planar_flow)

hub_map.save(f'{plot_folder}/paper_planar_overview.html')

[[-73.983313    40.733348     0.8660254    0.5         -0.97908409
    0.20345601]]


In [84]:
combi_flow_path = os.path.join(root_path, "results/nyc_taxi/combi/test_combi_fixed_bn/run3/nyc_model_combi.pickle")

with open(combi_flow_path, 'rb') as f:
    loaded_dict = pickle.load(f)

combi_flow = loaded_dict['model']



hub_map = plot_func(hour=input_hour, dayofweek=input_dayofweek, pickup_lat=input_pickup_lat, pickup_lon=input_pickup_lon, flow=combi_flow)

hub_map.save(f'{plot_folder}/paper_combi_overview.html')

[[-73.983313    40.733348     0.8660254    0.5         -0.97908409
    0.20345601]]
